## PMLDL Midterm - Fall 21
## **Task 1**

You are given a working code that performs a classification task on a given dataset. The data contains 10 classes (airplne, automobile, bird, cat, dog deer, frog, horse, ship, truck).

The model produces unsatisfactory results which can be improved. Your task will be to improve the results by making the necessary changes to the code.

Link to dataset : https://drive.google.com/file/d/1vOQeC2H_9qSYXRDdtKf59QEkjUlBRiGj/view?usp=sharing

In [81]:
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader , Dataset
from PIL import Image
import torchvision
from torchvision import transforms
import torch.utils

In [82]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
!unzip 'drive/MyDrive/dataset-1.zip' 

Archive:  drive/MyDrive/dataset-1.zip
replace dataset-1/class_names.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [83]:
num_batches = 512
random_state = 43
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

def read_images(path_to_data):
    with open(path_to_data, 'rb') as f:
        everything = np.fromfile(f, dtype=np.uint8)
        images = np.reshape(everything, (-1, 3, 96, 96))
        images = np.transpose(images, (0, 3, 2, 1))
        return images

def read_labels(path_to_labels):
    with open(path_to_labels, 'rb') as f:
        labels = np.fromfile(f, dtype=np.uint8)
        return labels

class CustomDataset(Dataset):
    def __init__(self, images, labels,transforms=None):
        self.labels = labels
        self.images = images 
        self.transform = transforms
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx] - 1
        data = self.images[idx]
        if self.transform is not None:
            return (self.transform(data),label)
        else:
            return (data,label)

X_train = read_images('dataset-1/train_X.bin')  # Changed the path as it was not working (original: stl10_binary)
X_test = read_images('dataset-1/test_X.bin')
y_train = read_labels('dataset-1/train_y.bin')
y_test = read_labels('dataset-1/test_y.bin')


train_data = CustomDataset(X_train,y_train,train_transform)
train_loader = DataLoader(train_data,batch_size=num_batches,shuffle=True)

test_data =   CustomDataset(X_test,y_test,test_transform)
test_loader = DataLoader(test_data,batch_size=num_batches,shuffle=True)

torch.manual_seed(random_state)

print("Train data %.d  Test data %.d "%(len(train_loader.dataset),len(test_loader.dataset)))


Train data 5000  Test data 8000 


In [84]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

NB_CLASS = 10
class CNN_model(nn.Module):
    def __init__(self,nb_class):
        super(CNN_model, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, 3)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, 3)
        self.bn3 = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(2, 2)

        self.dropout = nn.Dropout(0.2)


        self.fc1 = nn.Linear(12800, 4096)
        self.fc2 = nn.Linear(4096, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, nb_class)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(F.relu(self.bn1(x)))
        # x = self.dropout(x)

        x = self.conv2(x)
        x = self.pool(F.relu(self.bn2(x)))

        x = self.conv3(x)
        x = self.pool(F.relu(self.bn3(x)))
        # x = self.dropout(x)

        x = torch.flatten(x, 1)
        # print(x.shape)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        x = F.relu(self.fc2(x))
        x = self.dropout(x)

        x = F.relu(self.fc3(x))
        x = (self.fc4(x))
        return x


model = CNN_model(NB_CLASS)
model = model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.002)
scheduler = None
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

In [85]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

def train(model,train_loader,loss_fn,optimizer,scheduler,device):
    accuracy = 0
    running_loss = 0
    total = 0
    correct = 0
    model.train()
    for i, data in enumerate(train_loader):
        inputs, labels = data
        images,labels = inputs.to(device) , labels.to(device) 
        optimizer.zero_grad()
        outputs= model(images)
        loss = loss_fn(outputs, labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()* images.size(0)
        total += images.size(0)
        preds = torch.argmax(outputs.data, 1)
        correct += (preds == labels).sum().item()
        if i % 300 == 0:   
            print('[%d, %5d] Acc: %.3f loss: %.3f' %
                  (epoch + 1, i + 1, correct/total, running_loss /total))
            running_loss = 0.0
        # scheduler.step(loss/total)

def evaluate(model, loader, criterion, device):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for (images, labels) in loader:
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            loss = criterion(output, labels)
            acc = calculate_accuracy(output, labels)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    
    print("Epoch %d Test Accuracy %.3f" % (epoch, epoch_acc/ len(loader)))

In [86]:
for epoch in range(70):
    train(model,train_loader,loss_fn,optimizer,scheduler,device)
    evaluate(model,test_loader,loss_fn,device)

[1,     1] Acc: 0.100 loss: 2.304
Epoch 0 Test Accuracy 0.123
[2,     1] Acc: 0.131 loss: 2.257
Epoch 1 Test Accuracy 0.197
[3,     1] Acc: 0.230 loss: 2.105
Epoch 2 Test Accuracy 0.296
[4,     1] Acc: 0.252 loss: 1.957
Epoch 3 Test Accuracy 0.281
[5,     1] Acc: 0.322 loss: 1.789
Epoch 4 Test Accuracy 0.357
[6,     1] Acc: 0.318 loss: 1.718
Epoch 5 Test Accuracy 0.384
[7,     1] Acc: 0.383 loss: 1.640
Epoch 6 Test Accuracy 0.394
[8,     1] Acc: 0.383 loss: 1.599
Epoch 7 Test Accuracy 0.429
[9,     1] Acc: 0.445 loss: 1.460
Epoch 8 Test Accuracy 0.436
[10,     1] Acc: 0.457 loss: 1.444
Epoch 9 Test Accuracy 0.441
[11,     1] Acc: 0.500 loss: 1.397
Epoch 10 Test Accuracy 0.479
[12,     1] Acc: 0.445 loss: 1.349
Epoch 11 Test Accuracy 0.486
[13,     1] Acc: 0.518 loss: 1.265
Epoch 12 Test Accuracy 0.510
[14,     1] Acc: 0.541 loss: 1.133
Epoch 13 Test Accuracy 0.512
[15,     1] Acc: 0.600 loss: 1.109
Epoch 14 Test Accuracy 0.515
[16,     1] Acc: 0.525 loss: 1.190
Epoch 15 Test Accuracy 0